# Capstone Project Final - The Battle of Neighborhoods

### Introduction

New York City, the largest city in the United States with a population of around 8.5 million people, accounting for over 40% of the population of New York State. 
New York City has been a key tourist attraction for vistors from all over the world. Making it one of the most visted cites in the world. And its no coincidence that it also has some of the best and diverse resturants available to cater for people from all over the world. 

### Problem

Being one of the most visited cities in the world. This bring big opportunities for restaurateur to setup business in New York. However finding the best location to setup a Restaurant is a challenge with so many competiton.

Our goal would be to find the best location to setup a Thai Restaurant by looking at how the restaurants are clustered within the city and to find the best optimal location.


### Data

We will be using the below datasets to give us all the Borough and Neighborhoods in New York City

https://cocl.us/new_york_dataset

We will pull from FOURSQUARE to gather all the resturants information like location and cluster them for further analysis.

### Import necessary libraries

In [30]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import json
from pandas.io.json import json_normalize
import numpy as np

%matplotlib inline

In [2]:
url='https://cocl.us/new_york_dataset'
resp=requests.get(url).json()
neighborhoods_data=resp['features']

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
ny_neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    ny_neighborhoods = ny_neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [3]:
ny_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [25]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [4]:
CLIENT_ID = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [5]:
radius=1000
LIMIT=100

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue ID',         
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
ny_venues = getNearbyVenues(names=ny_neighborhoods['Neighborhood'],
                                   latitudes=ny_neighborhoods['Latitude'],
                                   longitudes=ny_neighborhoods['Longitude']
                                  )

In [20]:
print(ny_venues.shape)
ny_venues.head()

(20842, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,4c537892fd2ea593cb077a28,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,4d375ce799fe8eec99fd2355,40.898152,-73.838875,Caribbean Restaurant
2,Wakefield,40.894705,-73.847201,Rite Aid,4d6af9426107f04dedeb297a,40.896649,-73.844846,Pharmacy
3,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,4c10f6aece57c92804a682d2,40.889283,-73.843310,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Ali's Roti Shop,4c9e50e38afca09379b2ff15,40.894036,-73.856935,Caribbean Restaurant


In [26]:
ny_Restaurant = ny_venues[ny_venues['Venue Category'].str.contains('Restaurant')]
ny_Restaurant.reset_index()

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category
0,1,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,4d375ce799fe8eec99fd2355,40.898152,-73.838875,Caribbean Restaurant
1,3,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,4c10f6aece57c92804a682d2,40.889283,-73.843310,Caribbean Restaurant
2,4,Wakefield,40.894705,-73.847201,Ali's Roti Shop,4c9e50e38afca09379b2ff15,40.894036,-73.856935,Caribbean Restaurant
3,8,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,508af256e4b0578944c87392,40.898276,-73.850381,Caribbean Restaurant
4,15,Wakefield,40.894705,-73.847201,Golden Krust Caribbean Restaurant,4bf5c0a294b2a5937c42acee,40.892863,-73.857594,Caribbean Restaurant
5,19,Wakefield,40.894705,-73.847201,McDonald's,4be5f0eacf200f47d1fa133c,40.902645,-73.849485,Fast Food Restaurant
6,25,Wakefield,40.894705,-73.847201,McDonald's,4e4e0e38bd4101d0d7a12e00,40.891376,-73.851632,Fast Food Restaurant
7,26,Wakefield,40.894705,-73.847201,McDonald's,4bc256eb920eb713cd1b1c2c,40.892779,-73.857473,Fast Food Restaurant
8,33,Wakefield,40.894705,-73.847201,Hong Kong China King,4edbac4e775bcc53fc0ed5f8,40.891640,-73.840898,Chinese Restaurant
9,35,Wakefield,40.894705,-73.847201,New China Gardens,4e1627498877d38dc9037a11,40.898361,-73.853258,Asian Restaurant


In [7]:
ny_Restaurant.groupby(["Neighborhood"]).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Allerton,12,12,12,12,12,12,12
Annadale,2,2,2,2,2,2,2
Arden Heights,5,5,5,5,5,5,5
Arlington,3,3,3,3,3,3,3
Arrochar,4,4,4,4,4,4,4
Arverne,2,2,2,2,2,2,2
Astoria,28,28,28,28,28,28,28
Astoria Heights,7,7,7,7,7,7,7
Auburndale,32,32,32,32,32,32,32


In [8]:
print('There are {} uniques categories.'.format(len(ny_Restaurant['Venue Category'].unique())))

There are 94 uniques categories.


In [10]:
ny_Restaurant['Venue Category'].unique()

array(['Caribbean Restaurant', 'Fast Food Restaurant',
       'Chinese Restaurant', 'Asian Restaurant', 'Dumpling Restaurant',
       'Seafood Restaurant', 'Spanish Restaurant', 'Mexican Restaurant',
       'American Restaurant', 'Restaurant', 'Tapas Restaurant',
       'Greek Restaurant', 'Japanese Restaurant', 'Thai Restaurant',
       'Italian Restaurant', 'Latin American Restaurant',
       'Cuban Restaurant', 'Indian Restaurant',
       'Vegetarian / Vegan Restaurant', 'Sushi Restaurant',
       'Eastern European Restaurant', 'New American Restaurant',
       'French Restaurant', 'African Restaurant',
       'Middle Eastern Restaurant', 'Empanada Restaurant',
       'Vietnamese Restaurant', 'Paella Restaurant', 'Theme Restaurant',
       'Peruvian Restaurant', 'Southern / Soul Food Restaurant',
       'Arepa Restaurant', 'Mediterranean Restaurant',
       'Comfort Food Restaurant', 'Caucasian Restaurant',
       'Turkish Restaurant', 'Lebanese Restaurant',
       'Cantonese Restau

In [55]:
ny_Thai_Rest = ny_Restaurant[ny_Restaurant['Venue Category'] == 'Thai Restaurant']

In [58]:
ny_Thai_Rest

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category
55,Riverdale,40.890834,-73.912585,Nam Thai,4b94a4a2f964a520787f34e3,40.886211,-73.909991,Thai Restaurant
74,Kingsbridge,40.881687,-73.902818,Nam Thai,4b94a4a2f964a520787f34e3,40.886211,-73.909991,Thai Restaurant
85,Marble Hill,40.876551,-73.910660,Siam Square,4a739e29f964a520f5dc1fe3,40.878796,-73.916701,Thai Restaurant
337,Country Club,40.844246,-73.824099,Honey's Thai Pavilion,4b4c90d3f964a52046b626e3,40.847845,-73.831320,Thai Restaurant
351,Parkchester,40.837938,-73.856003,Thai No. 1,5585c948498e0bd8a8a706f1,40.838089,-73.863635,Thai Restaurant
388,Van Nest,40.843608,-73.866299,Thai No. 1,5585c948498e0bd8a8a706f1,40.838089,-73.863635,Thai Restaurant
452,Spuyten Duyvil,40.881395,-73.917190,Siam Square,4a739e29f964a520f5dc1fe3,40.878796,-73.916701,Thai Restaurant
454,Spuyten Duyvil,40.881395,-73.917190,Nam Thai,4b94a4a2f964a520787f34e3,40.886211,-73.909991,Thai Restaurant
471,Pelham Bay,40.850641,-73.832074,Honey's Thai Pavilion,4b4c90d3f964a52046b626e3,40.847845,-73.831320,Thai Restaurant
612,Bay Ridge,40.625801,-74.030621,Top Thai,50e8d45ae4b06202fbe39b08,40.623236,-74.028173,Thai Restaurant


In [77]:
kclusters = 7
Thai_clustering2 = ny_Thai_Rest.drop(["Neighborhood","Neighborhood Latitude","Neighborhood Longitude","Venue","Venue ID","Venue Category"], 1)

# run k-means clustering
kmeans2 = KMeans(n_clusters=kclusters, random_state=0).fit(Thai_clustering2)

# check cluster labels generated for each row in the dataframe
kmeans2.labels_[0:10]

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 3], dtype=int32)

In [78]:
ny_merged2 = ny_Thai_Rest.copy()

# add clustering labels
ny_merged2["Cluster Labels"] = kmeans2.labels_

In [79]:
ny_merged2

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels
55,Riverdale,40.890834,-73.912585,Nam Thai,4b94a4a2f964a520787f34e3,40.886211,-73.909991,Thai Restaurant,5
74,Kingsbridge,40.881687,-73.902818,Nam Thai,4b94a4a2f964a520787f34e3,40.886211,-73.909991,Thai Restaurant,5
85,Marble Hill,40.876551,-73.910660,Siam Square,4a739e29f964a520f5dc1fe3,40.878796,-73.916701,Thai Restaurant,5
337,Country Club,40.844246,-73.824099,Honey's Thai Pavilion,4b4c90d3f964a52046b626e3,40.847845,-73.831320,Thai Restaurant,5
351,Parkchester,40.837938,-73.856003,Thai No. 1,5585c948498e0bd8a8a706f1,40.838089,-73.863635,Thai Restaurant,5
388,Van Nest,40.843608,-73.866299,Thai No. 1,5585c948498e0bd8a8a706f1,40.838089,-73.863635,Thai Restaurant,5
452,Spuyten Duyvil,40.881395,-73.917190,Siam Square,4a739e29f964a520f5dc1fe3,40.878796,-73.916701,Thai Restaurant,5
454,Spuyten Duyvil,40.881395,-73.917190,Nam Thai,4b94a4a2f964a520787f34e3,40.886211,-73.909991,Thai Restaurant,5
471,Pelham Bay,40.850641,-73.832074,Honey's Thai Pavilion,4b4c90d3f964a52046b626e3,40.847845,-73.831320,Thai Restaurant,5
612,Bay Ridge,40.625801,-74.030621,Top Thai,50e8d45ae4b06202fbe39b08,40.623236,-74.028173,Thai Restaurant,3


In [80]:
# create map
map_newyork2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged2['Venue Latitude'], ny_merged2['Venue Longitude'], ny_merged2['Venue Category'], ny_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_newyork2)
       
map_newyork2

In [94]:
map_newyork2.save('Thai_clusters.html')

In [102]:
for cl in range(kclusters):
    print("There are " + str(len(ny_merged2[ny_merged2['Cluster Labels'] == cl])) + ' Thai restaurants in cluster ' + str(cl) + '\n')

There are 35 Thai restaurants in cluster 0

There are 10 Thai restaurants in cluster 1

There are 37 Thai restaurants in cluster 2

There are 38 Thai restaurants in cluster 3

There are 6 Thai restaurants in cluster 4

There are 9 Thai restaurants in cluster 5

There are 4 Thai restaurants in cluster 6



### Observations:

Most of the Thai restaurants are concentrated at the heart of New York City in cluster 0, 2 and 3 and moderate numbers in cluster 1. Its no suprising clusters 4, 5 and 6 have the lowest number as they are further away from Manhatten. 

At first thought, for less competition it would mean choosing cluster 4,5 or 6 as the choice of setting up a Thai Restaurant. But given the fact that location plays an important role for visibility and accessibility as well as the population base and human traffic to support the business. Choosing the suburb area might not be the best choice.

There is also the consideration of rent being higher at popular locations. The best compromise would be to choose a location in between. That would mean cluster 1 as its not too far from Manhatten and the restaurant concentration in that area are much spread out.